# Random Forest:

In [ ]:
class RFDecisionTree:
    def __init__(self, max_depth=12, min_samples_split=20, min_samples_leaf=5, feature_indices=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.feature_indices = feature_indices
        self.root = None
    
    def is_leaf_node(self, node):
        return not isinstance(node, dict)

    def gini_impurity(self, y):
        """Use Gini instead of Entropy for speed"""
        if len(y) == 0:
            return 0
        counts = np.bincount(y)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def gini_gain(self, y, feature_vector, threshold):
        """Faster than entropy calculation"""
        left = feature_vector <= threshold
        right = feature_vector > threshold
        
        if np.sum(left) == 0 or np.sum(right) == 0:
            return -1
            
        n_left, n_right = np.sum(left), np.sum(right)
        n_total = len(y)
        
        gini_left = self.gini_impurity(y[left])
        gini_right = self.gini_impurity(y[right])
        current_gini = self.gini_impurity(y)
        
        gain = current_gini - (n_left/n_total * gini_left + n_right/n_total * gini_right)
        return gain

    def best_split(self, X, y):
        """Optimized split finding with fewer threshold candidates"""
        best_gain = -1
        best_feature = None
        best_threshold = None

        if self.feature_indices is not None:
            features = self.feature_indices
        else:
            features = range(X.shape[1])
            
        for feature in features:
            feature_values = X[:, feature]
            
            # Use percentiles instead of all unique values for speed
            if len(feature_values) > 100:
                thresholds = np.percentile(feature_values, [25, 50, 75])
            else:
                unique_values = np.unique(feature_values)
                if len(unique_values) > 10:
                    thresholds = np.percentile(unique_values, [33, 66])
                else:
                    thresholds = unique_values[:-1]  # All except last
            
            for threshold in thresholds:
                gain = self.gini_gain(y, feature_values, threshold)
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        
        return best_feature, best_threshold, best_gain

    def build_tree(self, X, y, depth=0):
        n_samples = X.shape[0]

        # Early stopping conditions
        if (depth >= self.max_depth or 
            n_samples < self.min_samples_split or 
            len(np.unique(y)) == 1 or
            self.gini_impurity(y) < 0.01):  # Stop if already pure
            return self.most_common_label(y)
        
        feature, threshold, gain = self.best_split(X, y)
        
        if gain <= 0 or feature is None:  # No improvement
            return self.most_common_label(y)

        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        # Check minimum samples in leaves
        if (np.sum(left_mask) < self.min_samples_leaf or 
            np.sum(right_mask) < self.min_samples_leaf):
            return self.most_common_label(y)
        
        left_subtree = self.build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self.build_tree(X[right_mask], y[right_mask], depth + 1)
        
        return {
            'feature_index': feature, 
            'threshold': threshold, 
            'left': left_subtree, 
            'right': right_subtree
        }

    def most_common_label(self, y):
        if len(y) == 0:
            return 0
        return Counter(y).most_common(1)[0][0]

    def fit(self, X, y):
        self.root = self.build_tree(X, y)
        return self

    def predict_single(self, x, node):
        if self.is_leaf_node(node):
            return node
            
        if x[node['feature_index']] <= node['threshold']:
            return self.predict_single(x, node['left'])
        else:
            return self.predict_single(x, node['right'])

    def predict(self, X):
        return np.array([self.predict_single(x, self.root) for x in X])

In [ ]:
class RandomForestClassifier:
    def __init__(self, n_estimators=50, max_depth=12, min_samples_split=20, min_samples_leaf=5):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.trees = []
        self.n_classes = None

    def bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        
        n_samples, n_features = X.shape
        self.n_classes = len(np.unique(y))
        
        self.n_feature_samples = int(np.sqrt(n_features))
        
        self.trees = []
        
        print(f"Training Fast Random Forest with {self.n_estimators} trees...")
        print(f"Features per tree: {self.n_feature_samples}, Max depth: {self.max_depth}")
        
        for i in range(self.n_estimators):
            # Bootstrap sampling
            X_bootstrap, y_bootstrap = self.bootstrap_sample(X, y)
            
            # Feature sampling
            feature_indices = np.random.choice(n_features, self.n_feature_samples, replace=False)

            # Create and train tree with optimized parameters
            tree = RFDecisionTree(
                max_depth=self.max_depth, 
                min_samples_split=self.min_samples_split, 
                min_samples_leaf=self.min_samples_leaf,
                feature_indices=feature_indices
            )
            tree.fit(X_bootstrap, y_bootstrap)
            self.trees.append(tree)
            
            # Progress tracking
            if (i + 1) % 10 == 0:
                print(f"Trained tree {i + 1}/{self.n_estimators}")
        
        print("Random Forest training completed!")
        return self

    def predict(self, X):        
        X = np.asarray(X)

        # Batch prediction for efficiency
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        
        # Majority voting
        final_predictions = []
        for sample_idx in range(X.shape[0]):
            sample_predictions = tree_predictions[:, sample_idx]
            majority_vote = Counter(sample_predictions).most_common(1)[0][0]
            final_predictions.append(majority_vote)
        
        return np.array(final_predictions)

In [ ]:
# HYPERPARAMETER CONFIGURATIONS FOR 5-MINUTE TRAINING:

class VeryFastRF(RandomForestClassifier):
    """Fastest training (~2-3 minutes) - Good for testing"""
    def __init__(self):
        super().__init__(
            n_estimators=30,           # Fewer trees
            max_depth=10,              # Shallower trees
            min_samples_split=30,      # Prevent over-splitting
            min_samples_leaf=10,       # Larger leaves
            max_features='sqrt'        # Feature sampling
        )

class BalancedRF(RandomForestClassifier):
    """Best balance (~3-4 minutes) - RECOMMENDED"""
    def __init__(self):
        super().__init__(
            n_estimators=50,
            max_depth=12,
            min_samples_split=20,
            min_samples_leaf=5,
            max_features='sqrt'
        )

class AccurateRF(RandomForestClassifier):
    """Higher accuracy (~4-5 minutes)"""
    def __init__(self):
        super().__init__(
            n_estimators=70,
            max_depth=15,
            min_samples_split=15,
            min_samples_leaf=3,
            max_features='sqrt'
        )


# USAGE EXAMPLES:
def train_fast_random_forest():
    # Load your data
    # Xtrain, ytrain, Xval, yval = read_data(...)
    
    print(f"Training data shape: {Xtrain.shape}")
    
    # Choose based on your time constraints
    rf_model = BalancedRF()  # Recommended
    
    # Train
    rf_model.fit(Xtrain, ytrain)
    
    # Evaluate
    train_pred = rf_model.predict(Xtrain)
    val_pred = rf_model.predict(Xval)
    
    train_acc = np.mean(train_pred == ytrain)
    val_acc = np.mean(val_pred == yval)
    
    print(f"\nResults:")
    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Validation Accuracy: {val_acc:.4f}")
    
    return rf_model

In [ ]:
# Quick test with different configurations:
# print("Testing VeryFastRF (2-3 minutes):")
# model1 = VeryFastRF()
# model1.fit(Xtrain, ytrain)

# print("\nTesting BalancedRF (3-4 minutes):")
# model2 = BalancedRF()
# model2.fit(Xtrain, ytrain)

# print("\nTesting AccurateRF (4-5 minutes):")
# model3 = AccurateRF()
# model3.fit(Xtrain, ytrain)

In [ ]:
# for i in [model1, model2, model3]:
#     ypred = i.predict(Xval)
#     print(accuracy_score(yval, ypred))

# K-Means

In [ ]:
class KMeans:
    def __init__(self, n_clusters=10, max_iter=100, tol=1e-4, random_state=None):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        self.centroids = None
        self.labels_ = None
        self.cluster_to_label = None

    def _compute_distances(self, X, centroids):
        return np.sqrt(((X[:, np.newaxis, :] - centroids[np.newaxis, :, :]) ** 2).sum(axis=2))

    def fit(self, X, y=None):
        X = np.array(X, dtype=float)
        np.random.seed(self.random_state)
        n_samples, n_features = X.shape

        # Initialize centroids
        random_idxs = np.random.choice(n_samples, self.n_clusters, replace=False)
        self.centroids = X[random_idxs]

        for _ in range(self.max_iter):
            distances = self._compute_distances(X, self.centroids)
            labels = np.argmin(distances, axis=1)
            new_centroids = np.zeros((self.n_clusters, n_features))
            for i in range(self.n_clusters):
                members = X[labels == i]
                new_centroids[i] = members.mean(axis=0) if len(members) > 0 else self.centroids[i]
            shift = np.linalg.norm(self.centroids - new_centroids)
            self.centroids = new_centroids
            if shift < self.tol:
                break

        self.labels_ = np.argmin(self._compute_distances(X, self.centroids), axis=1)

        if y is not None:
            self.cluster_to_label = {}
            for c in range(self.n_clusters):
                members = y[self.labels_ == c]
                self.cluster_to_label[c] = Counter(members).most_common(1)[0][0] if len(members) > 0 else -1

        return self

    def predict(self, X):
        X = np.array(X, dtype=float)
        distances = self._compute_distances(X, self.centroids)
        return np.argmin(distances, axis=1)

    def predict_labels(self, X):
        if self.cluster_to_label is None:
            raise ValueError("Fit with y first to assign labels.")
        clusters = self.predict(X)
        return np.array([self.cluster_to_label[c] for c in clusters])
    
    def cluster_confidences(kmeans, y_true):
        confidences = {}
        for c in range(kmeans.n_clusters):
            members = y_true[kmeans.labels_ == c]
            if len(members) == 0:
                confidences[c] = 0
            else:
                majority_label, count = Counter(members).most_common(1)[0]
                confidences[c] = count / len(members)
        return confidences

# XGBoost

In [ ]:
class TreeNode:
    def __init__(self, feature_index=None, threshold=None, value=None, left=None, right=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right

In [ ]:
class XGBoostMultiClass:
    def __init__(self, n_estimators=50, learning_rate=0.1, max_depth=6, lambda_l2=1.0, gamma=0.1, 
                 min_child_weight=5, subsample=0.8, colsample_bytree=1.0):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.lambda_l2 = lambda_l2
        self.gamma = gamma
        self.min_child_weight = min_child_weight
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        self.trees = []  # List of lists: [class][tree]
        self.initial_predictions = None
        self.classes = None
        self.n_features = None

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def compute_gradients(self, y_true, scores):
        """Compute gradients and hessians for multi-class classification"""
        n_samples, n_classes = scores.shape
        probs = self.softmax(scores)
        
        y_one_hot = np.eye(n_classes)[y_true]
        gradients = probs - y_one_hot
        hessians = probs * (1 - probs)
        
        return gradients, hessians

    def gain(self, G, H, G_left, H_left, G_right, H_right):
        """Calculate split gain using XGBoost formula"""
        def calc_term(G, H):
            return (G ** 2) / (H + self.lambda_l2)
        
        gain = 0.5 * (calc_term(G_left, H_left) + calc_term(G_right, H_right) - calc_term(G, H)) - self.gamma
        return gain

    def best_split(self, X, gradients, hessians, feature):
        """Find best split for a single feature"""
        feature_values = X[:, feature]
        
        if len(feature_values) > 1000:
            thresholds = np.percentile(feature_values, [25, 50, 75])
        else:
            unique_vals = np.unique(feature_values)
            if len(unique_vals) > 10:
                thresholds = np.percentile(unique_vals, [20, 40, 60, 80])
            else:
                thresholds = unique_vals[:-1]
        
        best_gain = 0
        best_threshold = None
        
        G = np.sum(gradients)
        H = np.sum(hessians)
        
        for threshold in thresholds:
            left_mask = feature_values <= threshold
            right_mask = ~left_mask
            
            if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                continue
                
            G_left = np.sum(gradients[left_mask])
            H_left = np.sum(hessians[left_mask])
            G_right = G - G_left
            H_right = H - H_left

            if H_left < self.min_child_weight or H_right < self.min_child_weight:
                continue
            
            current_gain = self.gain(G, H, G_left, H_left, G_right, H_right)
            
            if current_gain > best_gain:
                best_gain = current_gain
                best_threshold = threshold
        
        return best_gain, best_threshold

    def build_tree(self, X, gradients, hessians, depth=0):
        """Build a single regression tree"""
        n_samples, n_features = X.shape
        
        # Stopping conditions
        if (depth >= self.max_depth or n_samples < 2 or np.sum(hessians) < self.min_child_weight):
            leaf_value = -np.sum(gradients) / (np.sum(hessians) + self.lambda_l2)
            return TreeNode(value=leaf_value)
        
        best_gain = 0
        best_feature = None
        best_threshold = None
        
        # Data subsampling
        if self.subsample < 1.0:
            subsample_size = int(self.subsample * n_samples)
            rows = np.random.choice(n_samples, subsample_size, replace=False)
            X, gradients, hessians = X[rows], gradients[rows], hessians[rows]
            n_samples = subsample_size

        features = range(n_features)

        for feature in features:
            gain, threshold = self.best_split(X, gradients, hessians, feature)
            
            if gain > best_gain and threshold is not None:
                best_gain = gain
                best_feature = feature
                best_threshold = threshold
        
        if best_gain == 0:
            leaf_value = -np.sum(gradients) / (np.sum(hessians) + self.lambda_l2)
            return TreeNode(value=leaf_value)
        
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask
        
        left_node = self.build_tree(X[left_mask], gradients[left_mask], hessians[left_mask], depth + 1)
        right_node = self.build_tree(X[right_mask], gradients[right_mask], hessians[right_mask], depth + 1)
        
        return TreeNode(feature_index=best_feature, threshold=best_threshold, left=left_node, right=right_node)

    def predict_tree(self, x, node):
        """Predict using a single tree"""
        if node.value is not None:
            return node.value
        
        if node.feature_index < len(x) and x[node.feature_index] <= node.threshold:
            return self.predict_tree(x, node.left)
        else:
            return self.predict_tree(x, node.right)

    def fit(self, X, y, X_val=None, y_val=None):
        """Train the XGBoost model without early stopping"""
        X = np.asarray(X)
        y = np.asarray(y)

        # --- Make sure labels start at 0 ---
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_samples, self.n_features = X.shape

        # Remap y to 0..n_classes-1
        class_to_index = {cls: idx for idx, cls in enumerate(self.classes)}
        y_mapped = np.array([class_to_index[val] for val in y])

        print(f"Training XGBoost on {n_samples} samples, {self.n_features} features, {n_classes} classes")

        # Initialize class probabilities
        class_counts = np.bincount(y_mapped, minlength=n_classes)
        class_probs = np.clip(class_counts / n_samples, 1e-15, 1 - 1e-15)
        self.initial_predictions = np.log(class_probs)

        # Initialize scores for training
        train_scores = np.tile(self.initial_predictions, (n_samples, 1))

        self.trees = [[] for _ in range(n_classes)]

        for iteration in range(self.n_estimators):
            gradients, hessians = self.compute_gradients(y_mapped, train_scores)

            # Train trees for each class
            for class_idx in range(n_classes):
                tree = self.build_tree(X, gradients[:, class_idx], hessians[:, class_idx])
                self.trees[class_idx].append(tree)

                # Update training scores
                tree_pred_train = np.array([self.predict_tree(x, tree) for x in X])
                train_scores[:, class_idx] += self.learning_rate * tree_pred_train

            # Print progress
            train_probs = self.softmax(train_scores)
            train_pred = np.argmax(train_probs, axis=1)
            train_acc = np.mean(train_pred == y_mapped)

            if X_val is not None and y_val is not None and iteration % 5 == 0:
                val_pred = self.predict(X_val)
                val_acc = np.mean(val_pred == y_val)
                print(f"Iteration {iteration}: Train acc = {train_acc:.4f}, Val acc = {val_acc:.4f}")
            else:
                print(f"Iteration {iteration}: Train accuracy = {train_acc:.4f}")

        print(f"Training completed with {self.n_estimators} iterations")
        return self


    def predict_proba(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]
        n_classes = len(self.classes)
        scores = np.full((n_samples, n_classes), self.initial_predictions)

        for class_idx in range(n_classes):
            for tree in self.trees[class_idx]:
                tree_pred = np.array([self.predict_tree(x, tree) for x in X])
                scores[:, class_idx] += self.learning_rate * tree_pred

        return self.softmax(scores)

    def predict(self, X):
        proba = self.predict_proba(X)
        class_indices = np.argmax(proba, axis=1)
        return self.classes[class_indices]

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions == y)

In [ ]:
# Test the fixed version
# print("Testing XGBoost:")
# model = XGBoostMultiClass(
#     n_estimators=50,
#     learning_rate=0.15,
#     max_depth=4,
#     lambda_l2=0.5,
#     subsample=0.8
# )

# model.fit(Xtrain_pca, ytrain, Xval_pca, yval)

In [ ]:
# ypred = model.predict(Xtest_pca)
# print(f1_score(ytest, ypred, average='weighted'))

# Softmax Regression

In [ ]:
class SoftmaxRegression:
    def __init__(self, learning_rate=0.01, n_epochs=100, batch_size=100, lambda_reg=0.01):
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.lambda_reg = lambda_reg  # L2 regularization
        
        self.theta = None
        self.classes = None
        self.n_features = None
        self.loss_history = []

    def _add_bias(self, X):
        """Add bias column to feature matrix"""
        if X.ndim == 1:
            X = X[:, np.newaxis]
        return np.c_[np.ones((X.shape[0], 1)), X]

    def _softmax(self, z):
        """Softmax with numerical stability"""
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def _compute_loss(self, y_true, y_pred):
        """Cross-entropy loss with L2 regularization"""
        m = y_true.shape[0]
        cross_entropy = -np.sum(y_true * np.log(y_pred + 1e-15)) / m
        reg = (self.lambda_reg / (2 * m)) * np.sum(self.theta[1:] ** 2)
        return cross_entropy + reg

    def _one_hot_encode(self, y, n_classes):
        """Convert integer labels to one-hot encoding"""
        y_one_hot = np.zeros((len(y), n_classes))
        y_one_hot[np.arange(len(y)), y] = 1
        return y_one_hot

    def fit(self, X, y):
        """Train the softmax regression model"""
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes = np.unique(y)
        n_classes = len(self.classes)
        X_b = self._add_bias(X)
        m, n = X_b.shape
        y_one_hot = self._one_hot_encode(y, n_classes)

        self.theta = np.random.randn(n, n_classes) * 0.01
        n_batches = max(1, m // self.batch_size)

        for epoch in range(self.n_epochs):
            indices = np.random.permutation(m)
            X_shuffled = X_b[indices]
            y_shuffled = y_one_hot[indices]

            epoch_loss = 0
            for i in range(n_batches):
                start_idx = i * self.batch_size
                end_idx = start_idx + self.batch_size
                X_batch = X_shuffled[start_idx:end_idx]
                y_batch = y_shuffled[start_idx:end_idx]

                logits = np.dot(X_batch, self.theta)
                y_pred = self._softmax(logits)

                errors = y_pred - y_batch
                gradients = (1.0 / len(X_batch)) * np.dot(X_batch.T, errors)
                gradients[1:] += (self.lambda_reg / m) * self.theta[1:]

                self.theta -= self.learning_rate * gradients
                epoch_loss += self._compute_loss(y_batch, y_pred)

            self.loss_history.append(epoch_loss / n_batches)

        return self

    def predict_proba(self, X):
        """Predict class probabilities"""
        X_b = self._add_bias(np.asarray(X))
        logits = np.dot(X_b, self.theta)
        return self._softmax(logits)

    def predict(self, X):
        """Predict class labels"""
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)

    def score(self, X, y):
        """Compute accuracy"""
        preds = self.predict(X)
        return np.mean(preds == y)

    def get_parameters(self):
        """Return model parameters"""
        return {
            'theta': self.theta,
            'classes': self.classes,
            'n_features': self.n_features,
            'loss_history': self.loss_history
        }

In [ ]:
# softmax = SoftmaxRegression(
#             learning_rate=0.05,
#             n_epochs=150,
#             batch_size=50,
#             lambda_reg=0.01,
#             verbose=True
#         )
# softmax.fit(Xtrain, ytrain)
# ypred = softmax.predict(Xtest)
# print(accuracy_score(ytest, ypred))

# Augmentations

In [ ]:
# !pip install albumentations

In [ ]:
import numpy as np
import albumentations as A

# Albumentations uses (H,W,C), so add a channel dimension later
augmenter = A.Compose([
    A.Rotate(limit=15, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Affine(shear=10, p=0.4),
    A.Perspective(scale=(0.02, 0.05), p=0.4),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.3)
])

def augment_data(X, y, multiplier=2):
    X = np.asarray(X)
    y = np.asarray(y)

    X_aug_list = []
    y_aug_list = []

    X_reshaped = X.reshape(-1, 28, 28)

    for _ in range(multiplier):
        images_aug = []
        for img in X_reshaped:
            aug = augmenter(image=img.astype(np.uint8))
            images_aug.append(aug["image"])
        X_aug_list.append(np.array(images_aug).reshape(len(X), -1))
        y_aug_list.append(y)

    X_final = np.vstack([X] + X_aug_list)
    y_final = np.hstack([y] + y_aug_list)
    return X_final, y_final

# Example:
# Xtrain_aug, ytrain_aug = augment_data(Xtrain, ytrain, multiplier=2)
# print("Augmented:", Xtrain_aug.shape, ytrain_aug.shape)

In [ ]:
# pca1 = PCA(n_components=120)
# Xtrain_aug_pca = pca1.fit_transform(Xtrain_aug)
# Xval_aug_pca = pca1.transform(Xval)